In [ ]:
from sympy import Matrix, Symbol, simplify, symbols


def build_functional_matrix(n, k, lambdas) -> Matrix:
    f = Matrix(symbols(",".join(f"f{j}" for j in range(1, n + 1))))
    gammas = symbols(",".join(f"\gamma{i}" for i in range(1, k + 1)))
    # lambdas = [-float(i + 1.) * 0.1 for i in range(1, k + 1)]
    # print(lambdas)
    result = Matrix.zeros(rows=n, cols=1)
    if k == 1:
        gammas = [gammas]
    for gamma, lambd in zip(gammas, lambdas):
        result += gamma * Matrix([lambd**j for j in range(n)])

    diff: Matrix = f - result
    return diff


def build_functional(n, k, lambdas):
    diff = build_functional_matrix(n, k, lambdas)
    return simplify(diff.norm() ** 2)


In [ ]:
import numpy as np
import plotly.graph_objects as go
from IPython.display import Math, display
from scipy.optimize import minimize
from sympy.printing import latex
from toolz import compose

mega_print = compose(display, Math) if False else print

n = 6

while True:
    f_vec = np.round(np.random.randint(-3, 3, n) + np.random.random(n), 2)
    # f_vec = np.array(
    #     [1.94714385, -2.06999027, -1.77805665, -1.47865507, 1.45323721, -1.59903824]
    # )
    # f_vec = np.array([0.5489, 0.0641, -0.3059, 0.8434, -0.853, -0.3352])
    # f_vec = np.array([1, -2, 1])
    # f_vec = restruct_f(f_vec)
    n = len(f_vec)
    # 6 2 10.226062424586425 gamma=[0.5823549  0.58430421] lambda=[-0.79111583 -1.17055549]
    # print(f_vec)
    mega_print(f"F={latex(Matrix(f_vec))}")
    index, function_result = [], []
    for k in range(1, n):
        lambdas = [-float(i + 1.0) * 0.1 for i in range(1, k + 1)]
        J = build_functional(n, k, lambdas)
        # display(J)
        expr = J.subs({Symbol(f"f{i}"): f_vec[i - 1] for i in range(1, n + 1)})

        def func(x):
            return float(
                expr.subs({Symbol(f"\gamma{i + 1}"): x[i] for i in range(0, k)}).evalf(
                    n=21
                )
            )

        b_gamma = [-100.0, 100.0]
        bounds = [b_gamma] * k

        x0 = [-1] * k
        solution = minimize(func, x0, bounds=bounds)  # method="L-BFGS-B", tol=1e-9)
        # print(lambdas)

        mega_print(f"k={k}", end=" & ")
        mega_print(f"\min{{J_{k}^{n}}}={np.round(solution.fun, 2)}", end=" & ")
        for i, x in enumerate(solution.x):
            mega_print(f"\gamma_{i}={np.round(x, 2)}", end=", ")
        print(" & ", end="")
        for i, x in enumerate(lambdas):
            mega_print(f"\lambda_{i + 1}={np.round(x, 2)}", end=", ")
        gramm_matrix = np.zeros((k, k))
        for i, l_i in enumerate(lambdas):
            li = np.array([l_i**k for k in range(n)])
            for j, l_j in enumerate(lambdas):
                lj = np.array([l_j**k for k in range(n)])
                gramm_matrix[i, j] = li @ lj
        print(" & ", end="")
        mega_print(
            f"eig(G_{k}^{n})={np.round(np.linalg.eigvals(gramm_matrix), 2)}",
            end="\\\\ \hline",
        )

        print()
        index.append(k)
        function_result.append(np.round(solution.fun, 2))

    fig = go.Figure(go.Scatter(x=index, y=function_result))
    fig.update_layout(
        xaxis_title="Значение 'k'",
        yaxis_title="min J",
        plot_bgcolor="white",
    )
    fig.update_xaxes(
        mirror=True,
        ticks="outside",
        showline=True,
        linecolor="black",
        gridcolor="lightgrey",
    )
    fig.update_yaxes(
        mirror=True,
        ticks="outside",
        showline=True,
        linecolor="black",
        gridcolor="lightgrey",
    )
    fig.show()


In [ ]:
import numpy as np
import plotly.graph_objects as go
from IPython.display import Math, display
from scipy.optimize import minimize
from sympy.printing import latex
from toolz import compose

mega_print = compose(display, Math) if False else print

n = 10
f_vec = np.round(np.random.randint(-3, 3, n) + np.random.random(n), 2)
f_vec = np.round(np.array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1]), 2)
coefs = (0.1, 0.4, 1, 10, 100)
fig = go.Figure(
    layout=dict(
        legend=dict(
            yanchor="top",
            y=-0.2,
            xanchor="left",
            x=0.01,
            orientation="h",
        )
    )
)
fig.update_layout(
    xaxis_title="Значение 'k'",
    yaxis_title="min J",
    plot_bgcolor="white",
)
fig.update_xaxes(
    mirror=True,
    ticks="outside",
    showline=True,
    linecolor="black",
    gridcolor="lightgrey",
)
fig.update_yaxes(
    mirror=True,
    ticks="outside",
    showline=True,
    linecolor="black",
    gridcolor="lightgrey",
)
for c in coefs:
    # f_vec = np.array(
    #     [1.94714385, -2.06999027, -1.77805665, -1.47865507, 1.45323721, -1.59903824]
    # )
    # f_vec = np.array([0.5489, 0.0641, -0.3059, 0.8434, -0.853, -0.3352])
    # f_vec = np.array([1, -2, 1])
    # f_vec = restruct_f(f_vec)
    n = len(f_vec)
    # 6 2 10.226062424586425 gamma=[0.5823549  0.58430421] lambda=[-0.79111583 -1.17055549]
    # print(f_vec)

    mega_print(f"F={tuple(f_vec)}")
    index, function_result = [], []
    for k in range(1, n - 1):
        lambdas = [-float(i + 1.0) * c for i in range(1, k + 1)]
        J = build_functional(n, k, lambdas)
        # display(J)
        expr = J.subs({Symbol(f"f{i}"): f_vec[i - 1] for i in range(1, n + 1)})

        def func(x):
            return float(
                expr.subs({Symbol(f"\gamma{i + 1}"): x[i] for i in range(0, k)}).evalf(
                    n=21
                )
            )

        b_gamma = [-10000.0, 10000.0]
        bounds = [b_gamma] * k

        x0 = [-1] * k

        gramm_matrix = np.zeros((k, k))
        b_vec = np.zeros(k)
        for i, l_i in enumerate(lambdas):
            li = np.array([l_i**k for k in range(n)])
            print(li)
            for j, l_j in enumerate(lambdas):
                lj = np.array([l_j**k for k in range(n)])
                gramm_matrix[i, j] = li @ lj
            b_vec[i] = li @ f_vec
        print(gramm_matrix)
        # print(b_vec)
        gammas = np.linalg.solve(gramm_matrix, b_vec)
        solution = func(gammas)
        # solution = minimize(func, x0, bounds=bounds)  # method="L-BFGS-B", tol=1e-9)
        # lambdas = [-float(i + 1.0) * 0.1 for i in range(1, k + 1)]
        # print(lambdas)
        # k=
        mega_print(f"{k}", end=" & ")
        # \min{{J_{k}^{n}}}=
        mega_print(f"{np.round(solution, 2)}", end=" & ")
        for i, x in enumerate(gammas):
            # \gamma_{i + 1}=
            mega_print(f"{np.round(x, 2)}", end=", ")
        # print(" & ", end="")
        # for i, x in enumerate(lambdas):
        #     mega_print(f"\lambda_{i + 1}={np.round(x, 2)}", end=", ")

        print("\\\\ \hline %", end="")
        mega_print(
            f"eig(G_{k}^{n})={np.linalg.eigvals(gramm_matrix)}",
            end="",
        )

        print()
        index.append(k)
        function_result.append(np.round(solution, 2))

    # fig = go.Figure()
    fig.add_trace(
        go.Scatter(x=index, y=function_result, name=str(np.round(lambdas, 3)))
    )

    fig.show()


In [ ]:
np.linalg.eig(
    [
        [
            2.62150554e41,
            1.00778640e43,
            1.34219406e44,
            1.00001000e45,
            5.15982335e45,
            2.06611944e46,
            6.87199062e46,
            1.98360392e47,
        ],
        [
            1.00778640e43,
            3.87424794e44,
            5.15982335e45,
            3.84436157e46,
            1.98360392e47,
            7.94283829e47,
            2.64181855e48,
            7.62562573e48,
        ],
        [
            1.34219406e44,
            5.15982335e45,
            6.87199062e46,
            5.12002560e47,
            2.64181855e48,
            1.05784937e49,
            3.51844820e49,
            1.01560239e50,
        ],
        [
            1.00001000e45,
            3.84436157e46,
            5.12002560e47,
            3.81471252e48,
            1.96830656e49,
            7.88158639e49,
            2.62144655e50,
            7.56682324e50,
        ],
        [
            5.15982335e45,
            1.98360392e47,
            2.64181855e48,
            1.96830656e49,
            1.01560239e50,
            4.06672352e50,
            1.35260828e51,
            3.90431314e51,
        ],
        [
            2.06611944e46,
            7.94283829e47,
            1.05784937e49,
            7.88158639e49,
            4.06672352e50,
            1.62841692e51,
            5.41617912e51,
            1.56338390e52,
        ],
        [
            6.87199062e46,
            2.64181855e48,
            3.51844820e49,
            2.62144655e50,
            1.35260828e51,
            5.41617912e51,
            1.80144267e52,
            5.19987700e52,
        ],
        [
            1.98360392e47,
            7.62562573e48,
            1.01560239e50,
            7.56682324e50,
            3.90431314e51,
            1.56338390e52,
            5.19987700e52,
            1.50094821e53,
        ],
    ]
)